In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import chart_studio.plotly as py
import plotly.offline as po
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
production=pd.read_excel('../data/1a - Total production.xlsx')
domestic_consumption = pd.read_excel('../data/1b - Domestic consumption.xlsx')
gross_opening_stocks = pd.read_excel('../data/1d - Gross Opening stocks.xlsx')
exports = pd.read_excel('../data/2a - Exports - calendar year.xlsx')
imports = pd.read_excel('../data/2b - Imports.xlsx')
re_exports = pd.read_excel('../data/2c - Re-exports.xlsx')
price_grower = pd.read_excel('../data/3a - Prices paid to growers.xlsx')
price_retail = pd.read_excel('../data/3b - Retail prices.xlsx')
consumption = pd.read_excel('../data/4b - Disappearance.xlsx')



## Data Cleaning

(1) Remove empty rows.

In [3]:
data = [production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption]
    


In [4]:
data_name = ['Production','Domestic_consumption','Gross_opening_stocks','Exports','Imports','Re_exports','Price_grower','Price_retail','Consumption']

In [5]:
for df in data:
    df = df.dropna(
        axis = 0,
        how = 'all',
        thresh = None,
        subset = None,
        inplace = True
    )

(2) Convert the format of crop year 'x/x+1' to 'x'. Rename the first column to 'Country'.

In [6]:
cropyearformat = [production,domestic_consumption,gross_opening_stocks]

In [7]:
cleaned = []

for df in cropyearformat:
    df = df.rename(columns = {x:x[:4] for x in df.columns})
    df = df.drop(columns=['Unna'])
    df = df.rename(columns = {'Crop':'Country'})
    df = df.reset_index()
    df = df.drop(columns=['index'])
    
    Apr = df["Country"] == "April group"
    Jul = df["Country"] == "July group"
    Oct = df["Country"] == "October group"
    Total = df["Country"] == "Total"
    
#     df_apr = df["Country"].iloc[df.index[Apr][0]:df.index[Jul][0]].values
#     df_jul = df["Country"].iloc[df.index[Jul][0]:df.index[Oct][0]].values
#     df_oct = df["Country"].iloc[df.index[Oct][0]:df.index[Total][0]].values
#     df_total = df["Country"].iloc[df.index[Total]].values
    
    df = df.drop([df.index[Apr][0],df.index[Jul][0],df.index[Oct][0],df.index[Total][0]],axis = 0)
    
    cleaned.append(df)

In [8]:
[production,domestic_consumption,gross_opening_stocks] = cleaned

In [9]:
for df in data:
    if 'Calendar years' in df.columns:
        df = df.rename(columns={'Calendar years':'Country'}, inplace = True)
         

(3) Remove Total

In [10]:
remove_total = [exports,imports,re_exports,consumption]

In [11]:
cleaned=[]
for df in remove_total:
    df = df.reset_index()
    df = df.drop(columns=['index'])
    Total = df["Country"] == "Total"
    df = df.drop([df.index[Total][0]],axis=0)
    cleaned.append(df)
exports,imports,re_exports,consumption = cleaned

(4) Remove European Union

In [12]:
remove_EU = [imports,re_exports,consumption,price_retail]

In [13]:
cleaned=[]
for df in remove_EU:
    df = df.reset_index()
    df = df.drop(columns=['index'])
    eu = df["Country"] == "European Union"
    df = df.drop([df.index[eu][0]],axis=0)
    cleaned.append(df)
imports,re_exports,consumption,price_retail = cleaned

(5) Milds info

In [14]:
price_grower.index

Int64Index([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
            19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36,
            37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
            54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
            71, 72],
           dtype='int64')

In [15]:
# remove if a row is empty except Country == %Milds
emptyrow = []
price_grower = price_grower.reset_index()
price_grower = price_grower.drop(columns=['index'])
for i in price_grower.index:
    if price_grower.iloc[i].notna().sum()==1:
        emptyrow.append(i)



In [16]:
price_grower = price_grower.drop(emptyrow,axis=0)

In [17]:
# unnamed column delete? what does it me? R/A or A? or A/R?
# unit specified later
# Note there are three groups: April, June, October. 

() Clean 'Country'

- strip()
- Malta, United Kingdom -> remove 1

In [18]:
data = [production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption]

In [19]:
cleaned=[]
for df in data:
    df["Country"] = df["Country"].str.strip()
    cleaned.append(df)

production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption=cleaned

In [20]:
price_retail.loc[price_retail["Country"] == 'United Kingdom 1']

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
27,United Kingdom 1,10.55,10.41,10.09,8.44,11.36,13.79,13.35,14.9,15.42,...,15.7,19.02,19.2,18.92,20.56,18.91,16.29,17.01,18.4,17.553314


In [21]:
price_retail  = price_retail.replace('Malta 1', 'Malta')

In [22]:
price_retail  = price_retail.replace('United Kingdom 1', 'United Kingdom')

Some of the column names are integer in 'price_grower' and 'price_retail. Convert them to string. 

In [23]:
price_grower.columns = price_grower.columns.astype(str)
price_retail.columns = price_retail.columns.astype(str)

### pd.melt to combine the tables

In [31]:
data = [production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption]
    


In [25]:
melted_data =[]
for ind, df in enumerate(data):
    df = pd.melt(df, id_vars = 'Country', value_vars = ['1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019'],
                        var_name = 'Year', value_name = data_name[ind])
    melted_data.append(df)

In [26]:
[production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption] = melted_data

### Merge data

In [32]:
coordinates_world=pd.read_csv('../data/coordinates_world.csv')

In [33]:
coordinates_world = coordinates_world.rename(columns={'country':'Country'})

production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption

In [34]:
right = exports

In [35]:
left = production

In [36]:
res = pd.merge(left, right, how = "outer", on = ["Country","Year"])

In [37]:
res

,Country,Year,Production,Exports
0,Angola,1990,50.3450,84.350000
1,Bolivia (Plurinational State of),1990,122.7770,156.442000
2,Brazil,1990,27285.6286,16935.787600
3,Burundi,1990,487.3930,584.773000
4,Ecuador,1990,1503.8150,1783.716000
...,...,...,...,...
1645,Trinidad & Tobago,2019,11.6910,2.471083
1646,Uganda,2019,5509.1540,4526.143000
1647,Venezuela,2019,650.1046,53.233395
1648,Viet Nam,2019,30487.1644,27400.161817


In [38]:
left = res
right = domestic_consumption
res = pd.merge(left, right, how = "outer",on = ["Country","Year"])

In [39]:
res

,Country,Year,Production,Exports,Domestic_consumption
0,Angola,1990,50.3450,84.350000,20.0
1,Bolivia (Plurinational State of),1990,122.7770,156.442000,25.0
2,Brazil,1990,27285.6286,16935.787600,8200.0
3,Burundi,1990,487.3930,584.773000,2.0
4,Ecuador,1990,1503.8150,1783.716000,350.0
...,...,...,...,...,...
1645,Trinidad & Tobago,2019,11.6910,2.471083,10.0
1646,Uganda,2019,5509.1540,4526.143000,254.0
1647,Venezuela,2019,650.1046,53.233395,1275.0
1648,Viet Nam,2019,30487.1644,27400.161817,2650.0


In [40]:
left = res
right = gross_opening_stocks
res = pd.merge(left, right,how = "outer", on = ["Country","Year"])

In [41]:
res

,Country,Year,Production,Exports,Domestic_consumption,Gross_opening_stocks
0,Angola,1990,50.3450,84.350000,20.0,210.000
1,Bolivia (Plurinational State of),1990,122.7770,156.442000,25.0,54.000
2,Brazil,1990,27285.6286,16935.787600,8200.0,26926.000
3,Burundi,1990,487.3930,584.773000,2.0,77.000
4,Ecuador,1990,1503.8150,1783.716000,350.0,723.963
...,...,...,...,...,...,...
1645,Trinidad & Tobago,2019,11.6910,2.471083,10.0,0.000
1646,Uganda,2019,5509.1540,4526.143000,254.0,900.000
1647,Venezuela,2019,650.1046,53.233395,1275.0,0.000
1648,Viet Nam,2019,30487.1644,27400.161817,2650.0,5200.000


In [42]:
left = res
right = imports
res = pd.merge(left, right,how="outer" ,on = ["Country","Year"])

In [43]:
res

,Country,Year,Production,Exports,Domestic_consumption,Gross_opening_stocks,Imports
0,Angola,1990,50.3450,84.3500,20.0,210.000,NaN
1,Bolivia (Plurinational State of),1990,122.7770,156.4420,25.0,54.000,NaN
2,Brazil,1990,27285.6286,16935.7876,8200.0,26926.000,NaN
3,Burundi,1990,487.3930,584.7730,2.0,77.000,NaN
4,Ecuador,1990,1503.8150,1783.7160,350.0,723.963,NaN
...,...,...,...,...,...,...,...
2695,Russian Federation,2019,NaN,NaN,NaN,NaN,5916.632578
2696,Switzerland,2019,NaN,NaN,NaN,NaN,3228.382979
2697,Tunisia,2019,NaN,NaN,NaN,NaN,508.332000
2698,United Kingdom,2019,NaN,NaN,NaN,NaN,5554.363879


In [44]:
left = res
right = re_exports
res = pd.merge(left, right,how="outer" ,on = ["Country","Year"])

In [47]:
left = res
right = price_grower
res = pd.merge(left, right,on = ["Country","Year"])

In [49]:
left = res
right = price_retail
res = pd.merge(left, right,how="outer" ,on = ["Country","Year"])

In [51]:
left = res
right = consumption
res = pd.merge(left, right,how="outer" ,on = ["Country","Year"])

In [53]:
left = res
right = coordinates_world[["country_code","latitude","longitude","Country"]]
res = pd.merge(left, right,how="left",on = ["Country"])

In [56]:
res['Year'] = pd.to_numeric(res['Year'])

In [57]:
res = res.rename(columns={'country_code':'Country_code',"latitude":"Latitude", "longitude":"Longitude"})

In [59]:
res.to_csv("../data/coffee-cleaned.csv", index=False)

In [84]:
cleaned = pd.read_csv("../data/coffee-cleaned.csv")

In [85]:
cleaned.head()

,Country,Year,Production,Exports,Domestic_consumption,Gross_opening_stocks,Imports,Re_exports,Price_grower,Price_retail,Consumption,Country_code,Latitude,Longitude
0,Angola,1990,50.3450,84.3500,20.0,210.0,NaN,NaN,NaN,NaN,NaN,AO,-11.202692,17.873887
1,Angola,1990,50.3450,84.3500,20.0,210.0,NaN,NaN,85.6569,NaN,NaN,AO,-11.202692,17.873887
2,Bolivia (Plurinational State of),1990,122.7770,156.4420,25.0,54.0,NaN,NaN,51.7802,NaN,NaN,NaN,NaN,NaN
3,Brazil,1990,27285.6286,16935.7876,8200.0,26926.0,NaN,NaN,54.3248,NaN,NaN,BR,-14.235004,-51.925280
4,Brazil,1990,27285.6286,16935.7876,8200.0,26926.0,NaN,NaN,34.8486,NaN,NaN,BR,-14.235004,-51.925280


In [86]:
data = [production,domestic_consumption,gross_opening_stocks,exports,imports,re_exports,price_grower,price_retail,consumption]

In [87]:
setofcountries = []

for df in data:
    setofcountries.append(df["Country"].unique())

In [88]:
prod,dome,gros,expo,impo,re_e,pgrower,pretail,cons = setofcountries

In [89]:
countries = cleaned.Country.unique()

In [90]:
newdf = pd.DataFrame(
        {
            'Country':countries,
            'inProduction':[False]*len(countries),
            'inDomestic_consumption':[False]*len(countries),
            'inGross_opening_stocks':[False]*len(countries),
            'inExports':[False]*len(countries),
            'inImports':[False]*len(countries),
            'inRe_exports':[False]*len(countries),
            'inPrice_grower':[False]*len(countries),
            'inPrice_retail':[False]*len(countries),
            'inConsumption':[False]*len(countries)
        
            
        }
)

In [91]:
for i in newdf.index:
    if newdf["Country"].iloc[i] in prod:
        newdf["inProduction"].iloc[i] = True        
    if newdf["Country"].iloc[i] in dome:
        newdf["inDomestic_consumption"].iloc[i] = True
    if newdf["Country"].iloc[i] in gros:
        newdf["inGross_opening_stocks"].iloc[i]= True    
    if newdf["Country"].iloc[i] in expo:
        newdf["inExports"].iloc[i]= True    
    if newdf["Country"].iloc[i] in impo:
        newdf["inImports"].iloc[i]= True    
    if newdf["Country"].iloc[i] in re_e:
        newdf["inRe_exports"].iloc[i]= True    
    if newdf["Country"].iloc[i] in pgrower:
        newdf["inPrice_grower"].iloc[i]= True    
    if newdf["Country"].iloc[i] in pretail:
        newdf["inPrice_retail"].iloc[i]= True    
    if newdf["Country"].iloc[i] in cons:
        newdf["inConsumption"].iloc[i]= True    
    

/opt/homebrew/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [92]:
grouped = newdf.groupby(by = ['inProduction', 'inDomestic_consumption',
       'inGross_opening_stocks', 'inExports', 'inImports', 'inRe_exports',
       'inPrice_grower', 'inPrice_retail', 'inConsumption'])


In [93]:
group_map = {group: i for i, group in enumerate(grouped.groups.keys())}


In [94]:
newdf['group_id'] = newdf.apply(lambda x: group_map[tuple(x[['inProduction', 'inDomestic_consumption', 'inGross_opening_stocks', 'inExports', 'inImports', 'inRe_exports', 'inPrice_grower', 'inPrice_retail', 'inConsumption']])], axis=1)

In [95]:
grouped.count()

Country  \
inProduction inDomestic_consumption inGross_opening_stocks inExports inImports inRe_exports inPrice_grower inPrice_retail inConsumption            
False        False                  False                  False     True      True         False          False          True                 7   
                                                                                                           True           True                28   
True         True                   True                   True      False     False        True           False          False               45   

                                                                                                                                         group_id  
inProduction inDomestic_consumption inGross_opening_stocks inExports inImports inRe_exports inPrice_grower inPrice_retail inConsumption            
False        False                  False                  False     True      True         False          False          True                  7  
                                                                                                           True           True                 28  
True         True                   True                   True      False     False        True           False          False                45

In [96]:
newdf['group_id'].value_counts()

2    45
1    28
0     7
Name: group_id, dtype: int64